In [1]:
import pandas as pd
import numpy as np

In [2]:
mutual_art = pd.read_csv('../datasets/clean/clean_mut_art_ver2_ajustado.csv').drop(columns=['Unnamed: 0'])
artsy = pd.read_csv('../datasets/clean/datosArtsy.csv')

In [3]:
def to_lower(df):
  columnsTolower = list(df.select_dtypes(include=['category','object']))
  for feature in columnsTolower:
      try:
          df[feature] = df[feature].str.lower()
      except:
          print('Error parsing '+feature)
  return df

In [4]:
artsy.height = artsy.height.str.split(' ')
artsy.length = artsy.length.str.split(' ')
artsy.height = artsy.height.apply(lambda x: x[0])
artsy.length = artsy.length.apply(lambda x: x[0])

In [5]:
# Transformación de tipo de datos
df = pd.concat([artsy,mutual_art])
df['height'] = df['height'].astype(float)
df['length'] = df['length'].astype(float)
df['price'] = df['price'].astype(float)
df['performance'] = df['performance'].astype(float)
#df['año_venta'] = pd.to_datetime(df['sale_date'],yearfirst=True)
#df['año_venta'] = df['año_venta'].astype(str).str.slice(stop=4)


# Añadir precio por cm2
df = df.assign(precio_cm2 = lambda x: (x['price']/(x['height']*x['length'])))
#Eliminar valores infinitos (Surgen cuando largo o ancho son 0)
df=df[~df.isin([np.inf, -np.inf]).any(1)]
df=to_lower(df)
df['author'] = df['author'].str.strip()

In [6]:
artistas_obras = df[['author','title']].groupby(['author']).agg({'title':['count']})
artistas_obras = artistas_obras['title']
artistas_obras = artistas_obras.sort_values(by='count',ascending=False)
artistas_obras = artistas_obras.reset_index()
lista_artistas = artistas_obras.author.head(20).to_list()
df_artistas = df[df.author.isin(lista_artistas)]
df_artistas['fecha_corta'] = df_artistas['fecha'].str.slice(0,7)+'-01'

/var/folders/c7/yrk77vqj3jj31wf8sfvg8tyw0000gn/T/ipykernel_67940/2754000025.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_artistas['fecha_corta'] = df_artistas['fecha'].str.slice(0,7)+'-01'


In [7]:
df_artistas

,title,height,length,art_type,medium_text,date_text,age,house,sale_date,price,author,performance,source,country,fecha,precio_cm2,fecha_corta
813,cafe caliente,38.000,46.000,painting,pastel and charcoal on canvas,1973.0,39.0,artcuria,1969-12-31 18:00:44.721,8436.43,antonio segui,0.120000,artsy,argentina,NaN,4.826333,NaN
814,untitled,62.000,45.000,work on paper,pastel on paper,1965.0,31.0,artcuria,1969-12-31 18:00:44.721,3374.79,antonio segui,0.050000,artsy,argentina,NaN,1.209602,NaN
815,textura city,114.000,146.000,painting,mixed technique and collage on canvas,1990.0,56.0,artcuria,1969-12-31 18:00:44.721,57048.79,antonio segui,1.480000,artsy,argentina,NaN,3.427589,NaN
816,barroco tropical,200.000,200.000,painting,oil on canvas,1984.0,50.0,sotheby's,1969-12-31 18:00:44.713,47339.64,antonio segui,0.100000,artsy,argentina,NaN,1.183491,NaN
817,untitled,100.000,81.000,painting,oil on canvas,NaN,NaN,sotheby's,1969-12-31 18:00:44.713,29756.35,antonio segui,0.580000,artsy,argentina,NaN,3.673623,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10464,femme aux escarpins verts,47.000,37.000,impressionist and modern art,huile sur toile,2001.0,21.0,sotheby's,NaN,325536.00,fernando botero,3.395455,mutual art,-,2008-12-03,187.197240,2008-12-01
10472,"no, no dice uno mientras otro duerme en el suelo",190.750,190.750,post-war & contemporary art,graphite and conté crayon and pastel on paper,1966.0,56.0,christie's,NaN,7716.00,roberto matta,0.750000,mutual art,-,2008-12-11,0.212062,2008-12-01
10480,las majas desnudas (dessin pour carmen ),28.000,38.100,impressionist & modern art day sale,"wax crayon, coloured crayon and pencil on paper",1940.0,82.0,sotheby's,NaN,37453.00,roberto matta,0.750000,mutual art,-,2009-02-04,35.107799,2009-02-01
10491,the new school,6.000,7.875,19th & 20th century prints & drawings,drypoint,1943.0,79.0,swann auction galleries,NaN,2319.00,roberto matta,1.120019,mutual art,-,2009-03-05,49.079365,2009-03-01


In [8]:
df_artistas = df_artistas[['author','fecha_corta','price']]

In [9]:
df_artistas

,author,fecha_corta,price
813,antonio segui,NaN,8436.43
814,antonio segui,NaN,3374.79
815,antonio segui,NaN,57048.79
816,antonio segui,NaN,47339.64
817,antonio segui,NaN,29756.35
...,...,...,...
10464,fernando botero,2008-12-01,325536.00
10472,roberto matta,2008-12-01,7716.00
10480,roberto matta,2009-02-01,37453.00
10491,roberto matta,2009-03-01,2319.00


In [10]:
df_artistas = df_artistas.groupby(['author','fecha_corta']).agg({'price':['mean']})

In [11]:
df_artistas = df_artistas['price'].reset_index()

In [12]:
df_artistas

,author,fecha_corta,mean
0,antonio seguí,1998-11-01,5543.000000
1,antonio seguí,1999-06-01,12037.000000
2,antonio seguí,2000-06-01,34355.000000
3,antonio seguí,2001-12-01,1574.000000
4,antonio seguí,2002-05-01,8221.000000
...,...,...,...
1469,rufino tamayo,2022-05-01,144891.166667
1470,rufino tamayo,2022-06-01,31993.538462
1471,rufino tamayo,2022-07-01,1369.500000
1472,rufino tamayo,2022-08-01,2760.000000


In [15]:
df_artistas_wide = df_artistas.pivot(index='fecha_corta', columns='author', values='mean').reset_index()

In [16]:
df_artistas_wide

author,fecha_corta,antonio seguí,david alfaro siqueiros,diego rivera,fernando botero,fernando de szyszlo,francisco toledo,francisco zuñiga,gunther gerzso,jesus-rafael soto,joaquín torres garcía,leonardo nierman,miguel covarrubias,rené portocarrero,roberto matta,rufino tamayo
0,1998-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102897.000000,NaN
1,1998-10-01,NaN,NaN,25073.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6059.666667
2,1998-11-01,5543.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49990.000000,NaN
3,1998-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6216.000000,12951.000000,NaN,NaN,6734.000,34536.000000,NaN
4,1999-05-01,NaN,NaN,NaN,13291.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,2022-05-01,8338.700000,39144.714286,61581.000000,229484.782609,37541.5,82017.0,34220.466667,NaN,32950.500000,36737.777778,NaN,340880.8,9069.875,67291.681818,144891.166667
241,2022-06-01,11869.666667,1399.500000,12608.333333,80672.400000,NaN,1547.0,2769.846154,NaN,11567.692308,50.000000,NaN,900.0,5163.400,7865.590909,31993.538462
242,2022-07-01,NaN,26805.000000,NaN,49958.666667,8190.0,3187.0,NaN,NaN,2141.000000,NaN,NaN,865.0,160.000,4460.000000,1369.500000
243,2022-08-01,210.000000,904.000000,NaN,1795.000000,1365.0,NaN,2879.000000,NaN,432.000000,NaN,NaN,NaN,NaN,728.000000,2760.000000


In [ ]:
df_artistas_wide.to_excel('')